## Queries check
We are generating a lot of queries for the main experiment. This notebook is a sense-check of that generation process.

In [18]:
import psycopg2
import boto3
import json
import pandas as pd

from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
# getting the secrets from the parameter store relating to the RDS database

session = boto3.Session()

ssm = session.client('ssm')

response = ssm.get_parameter(Name="/RAG/LABS_RDS_DB_CREDS", WithDecryption=True)

secret = response['Parameter']['Value']


In [4]:
conn = psycopg2.connect(**json.loads(secret))

In [29]:
cur = conn.cursor()

cur.execute("""
    SELECT DISTINCT tag
    FROM dbquery
""")

rows = cur.fetchall()

rows

[('whee',),
 ('test_queries_run_2',),
 ('test_queries_run_3',),
 ('test_09_aug_2',),
 ('local_run_1',),
 ('test-e2e-prefect-1',),
 ('test_queries_run',),
 ('test_main_queries1',),
 ('main_query_run_2024_08_10_sampled',),
 ('main_query_run_2024_08_10',),
 ('test',),
 ('local_test_prompts',),
 ('test_09_aug',),
 ('test_e2e_29072024',),
 ('main_run_21_08_2024_queries',),
 ('prompt-answer-experiment',),
 (None,),
 ('test_main_queries',),
 ('local_test',)]

In [30]:
cur = conn.cursor()

cur.execute("""
    SELECT *
    FROM dbquery
    WHERE tag = 'main_run_21_08_2024_queries'
""")

rows = cur.fetchall()

len(rows)

13976

In [33]:
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])[["text", "document_id", "prompt"]]

for p in df["prompt"].unique():
    print("="*50)
    print(p)
    print("\n- " + "\n- ".join(df[df["prompt"] == p]["text"].sample(10, random_state=42).tolist()))

main-experiment/queries-numerical

- What is the average number of employees per company in the industry?
- 2022
- What percentage of DFC's new investments will be focused on climate-related projects by fiscal year 2023?
- What percentage of the light vehicle fleet is expected to be zero emissions by 2050?
- how many families in Paraguay use firewood for cooking?
- What is the total amount of regionalization funds provided in the supplementary budget for 2022?
- What is the average annual direct loss due to natural disasters in Nepal in Nepali rupees?
- What is the average annual growth rate of the market?
- What is the average consumption of natural gas in Spain for homes with central heating?
- 20,000 megawatts
main-experiment/queries-search-terms

- Clean Energy Cess Rules 2010 registration
- GHG emissions reporting requirements Canada
- Djibouti energy commission mission
- impact on customer satisfaction
- Meteorological Services Vanuatu
- Antigua and Barbuda Emergency Powers Act
-

These mostly look okay, some of them seem a bit similar to each other. There are a lot of queries starting with "What is the ..." / "What are the ..."

In [32]:
df["document_id"].value_counts()

CCLW.executive.1534.1800         61
CCLW.executive.8612.1534         58
CCLW.executive.rtl_47.rtl_247    57
CCLW.executive.1714.2200         57
CCLW.executive.8646.rtl_90       53
                                 ..
CCLW.legislative.1776.2144       20
CCLW.legislative.1744.2172       19
CCLW.legislative.2021.2089       16
CCLW.executive.9541.3902         15
CCLW.document.i00000310.n0000     5
Name: document_id, Length: 550, dtype: int64

In [38]:
min(df["document_id"].value_counts(normalize=True).tolist()), max(df["document_id"].value_counts(normalize=True).tolist())

(0.0003577561534058386, 0.004364625071551231)

In [36]:
df["prompt"].value_counts(normalize=True) * 21

main-experiment/queries-normal              5.987765
main-experiment/queries-search-terms        2.425157
main-experiment/queries-no-answer           1.613766
main-experiment/queries-typo                1.179522
main-experiment/queries-speculative         1.176517
main-experiment/queries-sections            1.170507
main-experiment/queries-policy-violation    1.101388
main-experiment/queries-numerical           1.098383
main-experiment/queries-indigenous          1.009731
main-experiment/queries-long                1.002218
main-experiment/queries-harmful             0.814396
main-experiment/queries-controversial       0.809888
main-experiment/queries-bias-equity-risk    0.806883
main-experiment/queries-ambiguous           0.803878
Name: prompt, dtype: float64

In [39]:
df["text"].value_counts(sort=True, ascending=False).head(5)

hate speech                                                                                  94
indigenous land rights                                                                       79
climate justice                                                                              70
What is the average annual growth rate of the company's revenue over the past five years?    49
What are the key actions?                                                                    34
Name: text, dtype: int64

In [46]:
for t in df["text"].value_counts(sort=True, ascending=False).head(5).index:
    print(t)
    print(df[df["text"] == t]["prompt"].value_counts().to_dict())
    print("=" * 100)

hate speech
{'main-experiment/queries-harmful': 94}
indigenous land rights
{'main-experiment/queries-indigenous': 79}
climate justice
{'main-experiment/queries-bias-equity-risk': 69, 'main-experiment/queries-harmful': 1}
What is the average annual growth rate of the company's revenue over the past five years?
{'main-experiment/queries-numerical': 49}
What are the key actions?
{'main-experiment/queries-ambiguous': 34}


It looks like the generator is moderately creative in the case of the above categories, and goes on generating the same (mostly keywords). 